In [1]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}
.uplot{
   background-color: white;
}
</style>

In [2]:
import sys

sys.path.insert(0, "D:/my_repo/simple-quant/anywidget_demo/src")
sys.path.insert(0, "D:/my_repo/simple-quant/simple_quant")

In [3]:
from get_data import get_data, work_day, work_time, get_or_read_data
import datetime

demo_time = datetime.datetime.now()
# demo_time = datetime.datetime(2024, 3, 15, 14, 0)
d = work_day(demo_time)
t = work_time(demo_time)
print(demo_time, d, t)

2024-03-26 17:22:37.869975 True False


In [4]:
# get_data
import pandas as pd
from pathlib import Path

config = {
    "name": "SHFE.rb2405",
    # "name": "CZCE.MA405",
    # "name": "DCE.m2405",
    # "name": "CZCE.RM405",
    # "name": "DCE.c2405",
    # "name": "DCE.l2405",
    # "name": "CZCE.TA405",
    "frequency": "300s",  # 15s, 30s, 60s, 300s, 900s, 1800s
    "count": 6000,
    "refresh": True,
    # "end_time": None,
    "end_time": "2024-03-22 23:00:00",
}

path = Path(
    f"../../csv/{config['name']}/{config['name']} {config['frequency']} {config['count']} {config['end_time'].replace(':','_')}.csv"
)
path.parent.mkdir(parents=True, exist_ok=True)

data = get_or_read_data(config, path)
data

,time,open,high,low,close,volume,date
0,1.699845e+09,3881.0,3886.0,3878.0,3882.0,727,2023-11-13 11:15:00+08:00
1,1.699846e+09,3882.0,3889.0,3882.0,3887.0,1658,2023-11-13 11:20:00+08:00
2,1.699846e+09,3887.0,3891.0,3885.0,3886.0,1160,2023-11-13 11:25:00+08:00
3,1.699853e+09,3888.0,3892.0,3885.0,3890.0,2249,2023-11-13 13:30:00+08:00
4,1.699854e+09,3890.0,3892.0,3888.0,3890.0,1146,2023-11-13 13:35:00+08:00
...,...,...,...,...,...,...,...
5995,1.711118e+09,3610.0,3611.0,3605.0,3609.0,14903,2024-03-22 22:35:00+08:00
5996,1.711118e+09,3609.0,3612.0,3608.0,3611.0,11282,2024-03-22 22:40:00+08:00
5997,1.711119e+09,3611.0,3619.0,3609.0,3610.0,33380,2024-03-22 22:45:00+08:00
5998,1.711119e+09,3610.0,3611.0,3608.0,3610.0,14482,2024-03-22 22:50:00+08:00


In [5]:
import vectorbt as vbt
import numpy as np



price = data["close"]

windows=np.arange(2, 500, 5, dtype=int)

fast_ma, slow_ma = vbt.MA.run_combs(
    price, window=windows, r=2, short_names=["fast", "slow"]
)
fast_ma, slow_ma
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf_kwargs = dict(size=np.inf, fees=0.001, freq="1D")
pf = vbt.Portfolio.from_signals(price, entries, exits, **pf_kwargs)

fig = pf.total_return().vbt.heatmap(
    x_level="fast_window",
    y_level="slow_window",
    # slider_level="symbol",
    symmetric=True,
    trace_kwargs=dict(colorbar=dict(title="Total return", tickformat="%")),
)
fig.show()

d:\my_repo\simple-quant\.venv\lib\site-packages\jupyter_client\session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



In [6]:
np.arange(2, 500, 5, dtype=int)

array([  2,   7,  12,  17,  22,  27,  32,  37,  42,  47,  52,  57,  62,
        67,  72,  77,  82,  87,  92,  97, 102, 107, 112, 117, 122, 127,
       132, 137, 142, 147, 152, 157, 162, 167, 172, 177, 182, 187, 192,
       197, 202, 207, 212, 217, 222, 227, 232, 237, 242, 247, 252, 257,
       262, 267, 272, 277, 282, 287, 292, 297, 302, 307, 312, 317, 322,
       327, 332, 337, 342, 347, 352, 357, 362, 367, 372, 377, 382, 387,
       392, 397, 402, 407, 412, 417, 422, 427, 432, 437, 442, 447, 452,
       457, 462, 467, 472, 477, 482, 487, 492, 497])